# 1.7 Load Polygon Data into BigQuery

In this notbook we will confront the challenges we have faced in the form of *data limitations* by using the Polygon API in conjunction with the Google Cloud Service BigQuery. BigQuery supports terabyte and petabyte scale table queries so it should be a platform that is more than enough power for our needs. 

It will also form the backbone of our live data platform, once we transition to scale.

Using the script in pyscript, we queried the polygon API and stored the result as CSV files in a google storage bucket. We can now easily write them into BigQuery. 

In [1]:
from google.cloud import bigquery, storage

In [2]:
bq_client = bigquery.Client()
storage_client = storage.Client()

In [3]:
path = 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-06-23.csv'
df = pd.read_csv('gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-06-23.csv')
df.head()

,Unnamed: 0,interval_range,SYMBOL,open_timestamp,close_timestamp,size,open,low,high,close,dollar_volume
0,0,"(0.0, 1000000000.0]",A,2020-06-23 12:00:02.609093205,2020-06-23 12:00:02.609093205,6.0,88.53,88.53,88.53,88.53,5.311800e+02
1,1,"(0.0, 1000000000.0]",AAL,2020-06-23 08:00:00.052588358,2020-06-23 12:09:27.326002108,5801119.0,14.79,14.04,14.98,14.12,8.242122e+07
2,2,"(0.0, 1000000000.0]",AAP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"(0.0, 1000000000.0]",AAPL,2020-06-23 08:00:00.078834790,2020-06-23 12:09:26.875403317,405324.0,363.41,358.87,364.90,364.80,1.476195e+08
4,4,"(0.0, 1000000000.0]",ABBV,2020-06-23 08:07:45.983813848,2020-06-23 12:08:52.712542622,4731.0,98.25,97.73,99.33,98.48,4.656688e+05


In [4]:
df['interval_range'].dtype

dtype('O')

In [ ]:
error
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set table_id to the ID of the table to create.
# table_id = "your-project.your_dataset.your_table_name"

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("index_relic", "INTEGER"),
        bigquery.SchemaField("interval_range", "STRING"),
        bigquery.SchemaField("symbol", 'STRING'),
        bigquery.SchemaField("open_timestamp", 'TIMESTAMP'),
        bigquery.SchemaField("close_timestamp",'TIMESTAMP'),
        bigquery.SchemaField("size", 'FLOAT'),
        bigquery.SchemaField("open", 'FLOAT'),
        bigquery.SchemaField("low", "FLOAT"),
        bigquery.SchemaField("high", "FLOAT"),
        bigquery.SchemaField("close", "FLOAT"),
        bigquery.SchemaField("dollar_volume", "FLOAT")
    ],
    skip_leading_rows=1,
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
)
uri = path
table_id = "sp500historical.agg_1mm_bars"

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

In [19]:
error
uri = 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-06-24.csv'

job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
)

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 260075 rows.


In [51]:
uris = ['gs://fin-aml/' + blob.name for blob in storage_client.list_blobs('fin-aml', prefix='data/agg_data_pulls/gg')]
uris.pop(0)
uris.pop(0)
uris

['gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-06-25.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-06-26.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-06-29.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-06-30.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-01.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-02.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-06.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-07.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-08.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-09.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-10.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-13.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-14.csv']

In [5]:
uris = ['gs://fin-aml/' + blob.name for blob in storage_client.list_blobs('fin-aml', prefix='data/agg_data_pulls/gg')]

In [7]:
for i in range(15):
    uris.pop(0)
uris

['gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-15.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-16.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-17.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-20.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-21.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-22.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-23.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-24.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-27.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-28.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-29.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-30.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-07-31.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-08-03.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-08-04.csv',
 'gs://fin-aml/data/agg_data_pulls/gg_1m

In [52]:
table_id = "sp500historical.agg_1mm_bars"

job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
)

for uri in uris:
    load_job = client.load_table_from_uri(uri, table_id, job_config=job_config)
    
    load_job.result()
    destination_table = client.get_table(table_id)
    print("Loaded {} rows.".format(destination_table.num_rows))
    
print('Job complete')

Loaded 373195 rows.
Loaded 614585 rows.
Loaded 727200 rows.
Loaded 863045 rows.
Loaded 982730 rows.
Loaded 1097365 rows.
Loaded 1218060 rows.
Loaded 1332695 rows.
Loaded 1448845 rows.
Loaded 1567015 rows.
Loaded 1679125 rows.
Loaded 1818505 rows.
Loaded 1951825 rows.


In [9]:
table_id = "sp500historical.agg_1mm_bars"

job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
)

for uri in uris:
    load_job = bq_client.load_table_from_uri(uri, table_id, job_config=job_config)
    
    load_job.result()
    destination_table = bq_client.get_table(table_id)
    print("Loaded {} rows.".format(destination_table.num_rows))
    
print('Job complete')

Loaded 2082115 rows.
Loaded 2191195 rows.
Loaded 2312900 rows.
Loaded 2422485 rows.
Loaded 2537120 rows.
Loaded 2646200 rows.
Loaded 2771945 rows.
Loaded 2887590 rows.
Loaded 2995660 rows.
Loaded 3092620 rows.
Loaded 3201700 rows.
Loaded 3316840 rows.
Loaded 3498640 rows.
Loaded 3632465 rows.
Loaded 3753665 rows.
Loaded 3867290 rows.
Loaded 3982430 rows.
Loaded 4103630 rows.
Loaded 4228365 rows.
Loaded 4363200 rows.
Loaded 4477330 rows.
Loaded 4577320 rows.
Loaded 4668220 rows.
Loaded 4770230 rows.
Loaded 4872745 rows.
Loaded 4987885 rows.
Loaded 5089390 rows.
Loaded 5222710 rows.
Loaded 5350980 rows.
Loaded 5465110 rows.
Loaded 5605500 rows.
Job complete


In [59]:
df = pd.read_csv('gs://fin-aml/data/agg_data_pulls/gg_1mm_bars_2020-06-25.csv')
df.head()

,Unnamed: 0,interval_range,SYMBOL,open_timestamp,close_timestamp,size,open,low,high,close,dollar_volume
0,0,"(515000000000.0, 516000000000.0]",A,2020-06-25 11:37:51.743734311,2020-06-25 11:37:51.743734311,10.0,87.00,87.00,87.0000,87.0000,8.700000e+02
1,1,"(515000000000.0, 516000000000.0]",AAL,2020-06-24 23:58:53.133013062,2020-06-25 12:46:30.371060438,2348618.0,12.89,12.40,12.9700,12.4600,2.945387e+07
2,2,"(515000000000.0, 516000000000.0]",AAP,2020-06-25 12:09:07.505645782,2020-06-25 12:09:07.505645782,1.0,143.00,143.00,143.0000,143.0000,1.430000e+02
3,3,"(515000000000.0, 516000000000.0]",AAPL,2020-06-24 23:58:56.443933616,2020-06-25 12:46:30.928757703,363813.0,359.16,357.12,362.9796,360.7999,1.313293e+08
4,4,"(515000000000.0, 516000000000.0]",ABBV,2020-06-25 08:44:23.053477817,2020-06-25 12:44:59.734666939,35415.0,95.09,94.50,95.7800,94.9000,3.368999e+06


In [1]:
from google.cloud import bigquery
client = bigquery.Client()

In [4]:
from BqPivot.bq_pivot import BqPivot

gen = BqPivot(table_name='sp500historical.agg_1mm_bars',
              index_col="interval_range",
              pivot_col="SYMBOL",
              values_col=["size","open","close","high","low","dollar_volume"])

gen.write_query(verbose=False)

In [5]:
gen.query

'WITH wide_ranked AS ( \nSELECT ANY_VALUE(IF(SYMBOL = \'C\', rank, null)) as c_,\nANY_VALUE(IF(SYMBOL = \'F\', rank, null)) as f_,\nANY_VALUE(IF(SYMBOL = \'J\', rank, null)) as j_,\nANY_VALUE(IF(SYMBOL = \'O\', rank, null)) as o_,\nANY_VALUE(IF(SYMBOL = \'V\', rank, null)) as v_,\nANY_VALUE(IF(SYMBOL = \'BK\', rank, null)) as bk_,\nANY_VALUE(IF(SYMBOL = \'BR\', rank, null)) as br_,\nANY_VALUE(IF(SYMBOL = \'CB\', rank, null)) as cb_,\nANY_VALUE(IF(SYMBOL = \'CF\', rank, null)) as cf_,\nANY_VALUE(IF(SYMBOL = \'CL\', rank, null)) as cl_,\nANY_VALUE(IF(SYMBOL = \'DE\', rank, null)) as de_,\nANY_VALUE(IF(SYMBOL = \'ES\', rank, null)) as es_,\nANY_VALUE(IF(SYMBOL = \'GD\', rank, null)) as gd_,\nANY_VALUE(IF(SYMBOL = \'GL\', rank, null)) as gl_,\nANY_VALUE(IF(SYMBOL = \'GS\', rank, null)) as gs_,\nANY_VALUE(IF(SYMBOL = \'LW\', rank, null)) as lw_,\nANY_VALUE(IF(SYMBOL = \'PG\', rank, null)) as pg_,\nANY_VALUE(IF(SYMBOL = \'RF\', rank, null)) as rf_,\nANY_VALUE(IF(SYMBOL = \'UA\', rank, null))

In [6]:
gen.write_permanent_table(destination_table='tpu-training-289520.sp500historical.1mm_bars_pivoted')

NameError: name 'table_id' is not defined

In [7]:
print(gen.query)

WITH wide_ranked AS ( 
SELECT ANY_VALUE(IF(SYMBOL = 'C', rank, null)) as c_,
ANY_VALUE(IF(SYMBOL = 'F', rank, null)) as f_,
ANY_VALUE(IF(SYMBOL = 'J', rank, null)) as j_,
ANY_VALUE(IF(SYMBOL = 'O', rank, null)) as o_,
ANY_VALUE(IF(SYMBOL = 'V', rank, null)) as v_,
ANY_VALUE(IF(SYMBOL = 'BK', rank, null)) as bk_,
ANY_VALUE(IF(SYMBOL = 'BR', rank, null)) as br_,
ANY_VALUE(IF(SYMBOL = 'CB', rank, null)) as cb_,
ANY_VALUE(IF(SYMBOL = 'CF', rank, null)) as cf_,
ANY_VALUE(IF(SYMBOL = 'CL', rank, null)) as cl_,
ANY_VALUE(IF(SYMBOL = 'DE', rank, null)) as de_,
ANY_VALUE(IF(SYMBOL = 'ES', rank, null)) as es_,
ANY_VALUE(IF(SYMBOL = 'GD', rank, null)) as gd_,
ANY_VALUE(IF(SYMBOL = 'GL', rank, null)) as gl_,
ANY_VALUE(IF(SYMBOL = 'GS', rank, null)) as gs_,
ANY_VALUE(IF(SYMBOL = 'LW', rank, null)) as lw_,
ANY_VALUE(IF(SYMBOL = 'PG', rank, null)) as pg_,
ANY_VALUE(IF(SYMBOL = 'RF', rank, null)) as rf_,
ANY_VALUE(IF(SYMBOL = 'UA', rank, null)) as ua_,
ANY_VALUE(IF(SYMBOL = 'AAP', rank, null)) as aap_

In [28]:
import re

import pandas as pd


class BqPivot():
    """
    Class to generate a SQL query which creates pivoted tables in BigQuery.
    Example
    -------
    The following example uses the kaggle's titanic data. It can be found here -
    `https://www.kaggle.com/c/titanic/data`
    This data is only 60 KB and it has been used for a demonstration purpose.
    This module comes particularly handy with huge datasets for which we would need
    BigQuery(https://en.wikipedia.org/wiki/BigQuery).
    >>> from bq_pivot import BqPivot
    >>> import pandas as pd
    >>> data = pd.read_csv("titanic.csv").head()
    >>> gen = BqPivot(data=data, index_col=["Pclass", "Survived", "PassengenId"],
                      pivot_col="Name", values_col="Age",
                      add_col_nm_suffix=False)
    >>> print(gen.generate_query())
    select Pclass, Survived, PassengenId, 
    sum(case when Name = "Braund, Mr. Owen Harris" then Age else 0 end) as braund_mr_owen_harris,
    sum(case when Name = "Cumings, Mrs. John Bradley (Florence Briggs Thayer)" then Age else 0 end) as cumings_mrs_john_bradley_florence_briggs_thayer,
    sum(case when Name = "Heikkinen, Miss. Laina" then Age else 0 end) as heikkinen_miss_laina,
    sum(case when Name = "Futrelle, Mrs. Jacques Heath (Lily May Peel)" then Age else 0 end) as futrelle_mrs_jacques_heath_lily_may_peel,
    sum(case when Name = "Allen, Mr. William Henry" then Age else 0 end) as allen_mr_william_henry
    from <--insert-table-name-here-->
    group by 1,2,3
    
    """
    def __init__(self, index_col, pivot_col, values_col, agg_fun="sum", data=None, 
                 table_name=None, not_eq_default="0", add_col_nm_suffix=True, custom_agg_fun=None,
                 prefix=None, suffix=None):
        """
        Parameters
        ----------
        data: pandas.core.frame.DataFrame or string
            The input data can either be a pandas dataframe or a string path to the pandas
            data frame. The only requirement of this data is that it must have the column
            on which the pivot it to be done.
        index_col: list
            The names of the index columns in the query (the columns on which the group by needs to be performed)
        pivot_col: string
            The name of the column on which the pivot needs to be done.
        values_col: string or list of strings
            The name or names of the columns on which aggregation needs to be performed.
        agg_fun: string
            The name of the sql aggregation function.
        table_name: string
            The name of the table in the query.
        not_eq_default: numeric, optional
            The value to take when the case when statement is not satisfied. For example,
            if one is doing a sum aggregation on the value column then the not_eq_default should
            be equal to 0. Because the case statement part of the sql query would look like - 
            
            ... ...
            sum(case when <pivot_col> = <some_pivot_col_value> then values_col else 0)
            ... ...
            Similarly if the aggregation function is min then the not_eq_default should be
            positive infinity.
        add_col_nm_suffix: boolean, optional
            If True, then the original values column name will be added as suffix in the new 
            pivoted columns.
        custom_agg_fun: string, optional
            Can be used if one wants to give customized aggregation function. The values col name 
            should be replaced with {}. For example, if we want an aggregation function like - 
            sum(coalesce(values_col, 0)) then the custom_agg_fun argument would be - 
            sum(coalesce({}, 0)). 
            If provided this would override the agg_fun argument.
        prefix: string, optional
            A fixed string to add as a prefix in the pivoted column names separated by an
            underscore.
        suffix: string, optional
            A fixed string to add as a suffix in the pivoted column names separated by an
            underscore.        
        """
        if data == None and table_name == None:
            raise ValueError("At least one of data or table_name must be provided.")
            
        self.query = ""

        self.index_col = list(index_col)
        self.values_col = values_col
        self.pivot_col = pivot_col

        self.not_eq_default = not_eq_default
        
        
        if table_name:
            self.table_name = table_name
            self.piv_col_vals = self._query_piv_col_vals()
        elif data:
            self.piv_col_vals = self._get_piv_col_vals(data)
            self.table_name = self._get_table_name(table_name)
            
        if type(self.values_col) == str:
            self.piv_col_names = self._create_piv_col_names(add_col_nm_suffix, prefix, suffix)
        elif type(self.values_col) == list:
            self.piv_col_names = []
            for value_col in self.values_col:
                self.piv_col_names.append(self._create_piv_col_names(add_col_nm_suffix, prefix, suffix, value_col))
        
        self.function = custom_agg_fun if custom_agg_fun else agg_fun + "({})"

    def _get_table_name(self, table_name):
        """
        Returns the table name or a placeholder if the table name is not provided.
        """
        return table_name if table_name else "<--insert-table-name-here-->"
    
    def _query_piv_col_vals(self):
        return pd.read_gbq(f'SELECT DISTINCT({self.pivot_col}) FROM {self.table_name}')[self.pivot_col].astype(str).to_list()
    

    def _get_piv_col_vals(self, data):
        """
        Gets all the unique values of the pivot column.
        """
        if isinstance(data, pd.DataFrame):
            self.data = data
        elif isinstance(data, str):
            self.data = pd.read_csv(data)
        else:
            raise ValueError("Provided data must be a pandas dataframe or a csv file path.")

        if self.pivot_col not in self.data.columns:
            raise ValueError("The provided data must have the column on which pivot is to be done. "\
                             "Also make sure that the column name in the data is same as the name "\
                             "provided to the pivot_col parameter.")

        return self.data[self.pivot_col].astype(str).unique().tolist()
    
    def _clean_col_name(self, col_name):
        """
        The pivot column values can have arbitrary strings but in order to 
        convert them to column names some cleaning is required. This method 
        takes a string as input and returns a clean column name.
        """
        
        # replace spaces with underscores
        # remove non alpha numeric characters other than underscores
        # replace multiple consecutive underscores with one underscore
        # make all characters lower case
        # remove trailing underscores
        return re.sub("_+", "_", re.sub('[^0-9a-zA-Z_]+', '', re.sub(" ", "_", col_name))).lower().rstrip("_")

    def _create_piv_col_names(self, add_col_nm_suffix, prefix, suffix, value_col=None):
        """
        The method created a list of pivot column names of the new pivoted table.
        """
        prefix = prefix + "_" if prefix else ""
        suffix = "_" + suffix if suffix else ""
        if value_col == None:
            value_col = self.values_col

        if add_col_nm_suffix:
            piv_col_names = ["{0}{1}_{2}{3}".format(prefix, self._clean_col_name(piv_col_val), value_col.lower(), suffix)
                             for piv_col_val in self.piv_col_vals]
        else:
            piv_col_names = ["{0}{1}{2}".format(prefix, self._clean_col_name(piv_col_val), suffix)
                             for piv_col_val in self.piv_col_vals]

        return piv_col_names

    def _add_select_statement(self):
        """
        Adds the select statement part of the query.
        """
        query = "select " + "".join([index_col + ", " for index_col in self.index_col]) + "\n"
        return query

    def _add_case_statement(self):
        """
        Adds the case statement part of the query.
        """
        case_query = self.function.format("case when {0} = \"{1}\" then {2} else {3} end") + " as {4},\n"
        
        if type(self.values_col) == str:
            query = "".join([case_query.format(self.pivot_col, piv_col_val, self.values_col,
                                           self.not_eq_default, piv_col_name)
                         for piv_col_val, piv_col_name in zip(self.piv_col_vals, self.piv_col_names)])
            
        elif type(self.values_col) == list:
            query = ""
            
            for piv_col_names, value_col in zip(self.piv_col_names, self.values_col):
                query = query + "".join([case_query.format(self.pivot_col, piv_col_val, value_col,
                                                   self.not_eq_default, piv_col_name)
                                 for piv_col_val, piv_col_name in zip(self.piv_col_vals, self.piv_col_names)])
            
        
        query = query[:-2] + "\n"
        return query

    def _add_from_statement(self):
        """
        Adds the from statement part of the query.
        """
        query =  "from {0}\n".format(self.table_name)
        return query

    def _add_group_by_statement(self):
        """
        Adds the group by part of the query.
        """
        query = "group by " + "".join(["{0},".format(x) for x in range(1, len(self.index_col) + 1)])
        return query[:-1]

    def generate_query(self):
        """
        Returns the query to create the pivoted table.
        """
        self.query = self._add_select_statement() +\
                     self._add_case_statement() +\
                     self._add_from_statement() +\
                     self._add_group_by_statement()

        return self.query

    def write_query(self, output_file=None):
        """
        Writes the query to a text file if output_file is passed, or prints the query to the console.
        """
        if output_file == None:
            print(self.generate_query())
        else:
            text_file = open(output_file, "w")
            text_file.write(self.generate_query())
            text_file.close()
            
    def submit_query(self, **kwargs):
        '''
        Submits the query and returns the results. 
        '''
        if self.query == "":
            self.generate_query()
        return pd.read_gbq(self.query)

In [29]:
# add functionality to read directly from bgq

# 1. get the pivot col names directly from bgq 
# SELECT UNIQUE(pivot_col_name) FROM table

# 2. after writing the query, submit the query and read the result 



In [30]:
#data = df

gen = BqPivot(table_name='sp500historical.agg_1mm_bars',
              index_col=["interval_range"],
              pivot_col="SYMBOL",
              values_col=["size","open"])

In [31]:
gen.write_query(debug=True)

select interval_range, 
sum(case when SYMBOL = "A" then size else 0 end) as a_size,
sum(case when SYMBOL = "C" then size else 0 end) as c_size,
sum(case when SYMBOL = "D" then size else 0 end) as d_size,
sum(case when SYMBOL = "F" then size else 0 end) as f_size,
sum(case when SYMBOL = "J" then size else 0 end) as j_size,
sum(case when SYMBOL = "K" then size else 0 end) as k_size,
sum(case when SYMBOL = "L" then size else 0 end) as l_size,
sum(case when SYMBOL = "O" then size else 0 end) as o_size,
sum(case when SYMBOL = "T" then size else 0 end) as t_size,
sum(case when SYMBOL = "V" then size else 0 end) as v_size,
sum(case when SYMBOL = "BA" then size else 0 end) as ba_size,
sum(case when SYMBOL = "BK" then size else 0 end) as bk_size,
sum(case when SYMBOL = "BR" then size else 0 end) as br_size,
sum(case when SYMBOL = "CB" then size else 0 end) as cb_size,
sum(case when SYMBOL = "CE" then size else 0 end) as ce_size,
sum(case when SYMBOL = "CF" then size else 0 end) as cf_size,
sum(

In [91]:
gen.piv_col_names

[['a_price',
  'aal_price',
  'aap_price',
  'aapl_price',
  'abbv_price',
  'abc_price',
  'abmd_price',
  'abt_price',
  'acn_price',
  'adbe_price',
  'adi_price',
  'adm_price',
  'adp_price',
  'adsk_price',
  'aee_price',
  'aep_price',
  'aes_price',
  'afl_price',
  'aig_price',
  'aiv_price'],
 ['a_open',
  'aal_open',
  'aap_open',
  'aapl_open',
  'abbv_open',
  'abc_open',
  'abmd_open',
  'abt_open',
  'acn_open',
  'adbe_open',
  'adi_open',
  'adm_open',
  'adp_open',
  'adsk_open',
  'aee_open',
  'aep_open',
  'aes_open',
  'afl_open',
  'aig_open',
  'aiv_open'],
 ['a_close',
  'aal_close',
  'aap_close',
  'aapl_close',
  'abbv_close',
  'abc_close',
  'abmd_close',
  'abt_close',
  'acn_close',
  'adbe_close',
  'adi_close',
  'adm_close',
  'adp_close',
  'adsk_close',
  'aee_close',
  'aep_close',
  'aes_close',
  'afl_close',
  'aig_close',
  'aiv_close']]

In [92]:
gen.piv_col_vals

['A',
 'AAL',
 'AAP',
 'AAPL',
 'ABBV',
 'ABC',
 'ABMD',
 'ABT',
 'ACN',
 'ADBE',
 'ADI',
 'ADM',
 'ADP',
 'ADSK',
 'AEE',
 'AEP',
 'AES',
 'AFL',
 'AIG',
 'AIV']

In [93]:
gen.values_col

['price', 'open', 'close']

In [99]:
print(gen._add_case_statement())

sum(case when SYMBOL = "A" then price else 0 end) as a_price,
sum(case when SYMBOL = "AAL" then price else 0 end) as aal_price,
sum(case when SYMBOL = "AAP" then price else 0 end) as aap_price,
sum(case when SYMBOL = "AAPL" then price else 0 end) as aapl_price,
sum(case when SYMBOL = "ABBV" then price else 0 end) as abbv_price,
sum(case when SYMBOL = "ABC" then price else 0 end) as abc_price,
sum(case when SYMBOL = "ABMD" then price else 0 end) as abmd_price,
sum(case when SYMBOL = "ABT" then price else 0 end) as abt_price,
sum(case when SYMBOL = "ACN" then price else 0 end) as acn_price,
sum(case when SYMBOL = "ADBE" then price else 0 end) as adbe_price,
sum(case when SYMBOL = "ADI" then price else 0 end) as adi_price,
sum(case when SYMBOL = "ADM" then price else 0 end) as adm_price,
sum(case when SYMBOL = "ADP" then price else 0 end) as adp_price,
sum(case when SYMBOL = "ADSK" then price else 0 end) as adsk_price,
sum(case when SYMBOL = "AEE" then price else 0 end) as aee_price,
sum(

In [136]:
!pip install pandas-gbq -U

In [10]:
table_name = 'sp500historical.agg_1mm_bars'
col = 'SYMBOL'
query = f'SELECT DISTINCT({col}) FROM {table_name}'
query
#

'SELECT DISTINCT(SYMBOL) FROM sp500historical.agg_1mm_bars'

In [12]:
pivot_vals = pd.read_gbq(f'SELECT DISTINCT({col}) FROM {table_name}')

In [18]:
pivot_vals['SYMBOL'].astype(str).to_list()

['A',
 'C',
 'D',
 'F',
 'J',
 'K',
 'L',
 'O',
 'T',
 'V',
 'BA',
 'BK',
 'BR',
 'CB',
 'CE',
 'CF',
 'CI',
 'CL',
 'DD',
 'DE',
 'DG',
 'EA',
 'ED',
 'EL',
 'ES',
 'EW',
 'FB',
 'FE',
 'GD',
 'GE',
 'GL',
 'GM',
 'GS',
 'HD',
 'IP',
 'IR',
 'IT',
 'KO',
 'KR',
 'LB',
 'LH',
 'LW',
 'MA',
 'MO',
 'MS',
 'MU',
 'NI',
 'PG',
 'PH',
 'PM',
 'RE',
 'RF',
 'RL',
 'SO',
 'TT',
 'UA',
 'VZ',
 'WM',
 'WU',
 'WY',
 'AAL',
 'AAP',
 'ABC',
 'ABT',
 'ACN',
 'ADI',
 'ADM',
 'ADP',
 'AEE',
 'AEP',
 'AES',
 'AFL',
 'AIG',
 'AIV',
 'AIZ',
 'AJG',
 'ALB',
 'ALK',
 'ALL',
 'AMD',
 'AME',
 'AMP',
 'AMT',
 'AON',
 'AOS',
 'APA',
 'APD',
 'APH',
 'ARE',
 'ATO',
 'AVB',
 'AVY',
 'AWK',
 'AXP',
 'AZO',
 'BAC',
 'BAX',
 'BBY',
 'BDX',
 'BEN',
 'BIO',
 'BKR',
 'BLK',
 'BLL',
 'BMY',
 'BSX',
 'BWA',
 'BXP',
 'CAG',
 'CAH',
 'CAT',
 'CCI',
 'CCL',
 'CDW',
 'CFG',
 'CHD',
 'CLX',
 'CMA',
 'CME',
 'CMG',
 'CMI',
 'CMS',
 'CNC',
 'CNP',
 'COF',
 'COG',
 'COO',
 'COP',
 'CPB',
 'CRM',
 'CSX',
 'CTL',
 'CVS',
 'CVX'

In [20]:
# WITH wide_ranked AS (
#     SELECT ANY_VALUE(IF(class = 'A', rank, null))) as A,
#            ANY_VALUE(IF(class = 'B', rank, null))) as B,
#            ANY_VALUE(IF(class = 'C', rank, null))) as C,
#            ANY_VALUE(IF(class = 'D', rank, null))) as D
#     FROM (
#         SELECT "1" AS grouby_only_col,
#                 class,
#                 RANK() over (ORDER BY class) AS rank
#         FROM (
#             SELECT DISTINCT class
            
table_name = 'project.dataset.Table'
index_col = 'index'
pivot_col = 'class'
pivot_col_vals = ['A','B','C','D']
value_col = 'value'
            
def write_wide_ranked(pivot_col, pivot_col_vals, table_name):
    
    query = 'WITH wide_ranked AS ( \nSELECT '
    
    query = query + "".join(["ANY_VALUE(IF({} = '{}', rank, null)) as {},\n".format(pivot_col, 
                                                                                    pivot_col_val, 
                                                                                    pivot_col_val) 
                                for pivot_col_val in pivot_col_vals])
    
    query = query[:-2] +'\nFROM (\nSELECT "1" AS groupby_only_col,\n'
    query = query + f"{pivot_col},\nRANK() over (ORDER BY {pivot_col}) AS rank\nFROM (\nSELECT DISTINCT {pivot_col}\n"
    query = query + f"FROM `{table_name}`\n)\n)\nGROUP BY groupby_only_col\n),\n"
    
    return query 




def write_long_array_aggregated(index_col, pivot_col, pivot_col_vals, value_col, table_name):
    
    query = f'long_array_aggregated AS (\n SELECT {index_col},\n ARRAY_AGG({value_col} ORDER BY rank) AS {value_col}\n'
    query = query + f"FROM (\nSELECT ranked_classes_by_id.{index_col} AS {index_col},\n"
    query = query + f"ranked_classes_by_id.rank as rank,\n"
    query = query + f"source.{value_col} as {value_col}\n"
    query = query + f"FROM `{table_name}` as source\n"
    query = query + f"RIGHT JOIN (\nSELECT {index_col},\n {pivot_col},\n"
    query = query + f"rank() over (PARTITION BY {index_col} ORDER BY {pivot_col} as rank\n"
    query = query + f"FROM (\n SELECT DISTINCT class\n"
    query = query + f"FROM `{table_name}`)\n CROSS JOIN (\n SELECT DISTINCT {index_col}\n"
    query = query + f"FROM `{table_name}`)\n"
    query = query + f") as ranked_classes_by_id\n USING({index_col}, {pivot_col})\n)\nGROUP BY {index_col}\n)\n"
    
    return query

def write_table_join(index_col, pivot_col, pivot_col_vals, value_col, table_name):
    query = f"SELECT long_array_aggregated.{index_col}, \n"
    query = query + "".join(["Long_array_aggregated.{}[ordinal({})] as {},\n".format(value_col, pivot_col_val, pivot_col_val) 
                                          for pivot_col_val in pivot_col_vals])
    
    #f"long_array_aggregated.{value_col}[ordinal({value for value in values_col})] as {value for value in values_col}, \n"
    
    query = query[:-2] + f"\nfrom long_array_aggregated, wide_ranked"
    return query

#long_array_agg = write_long_array_aggregated(pivot_col, pivot_col_vals, index_col, value_col, table_name)

wide_ranked_query = write_wide_ranked(pivot_col, pivot_col_vals, table_name)
#print(wide_ranked_query)

long_array_agg = write_long_array_aggregated(index_col, pivot_col, pivot_col_vals, value_col, table_name)

table_join = write_table_join(index_col, pivot_col, pivot_col_vals, value_col, table_name)

print(wide_ranked_query + long_array_agg + table_join)

WITH wide_ranked AS ( 
SELECT ANY_VALUE(IF(class = 'A', rank, null)) as A,
ANY_VALUE(IF(class = 'B', rank, null)) as B,
ANY_VALUE(IF(class = 'C', rank, null)) as C,
ANY_VALUE(IF(class = 'D', rank, null)) as D
FROM (
SELECT "1" AS groupby_only_col,
class,
RANK() over (ORDER BY class) AS rank
FROM (
SELECT DISTINCT class
FROM `project.dataset.Table`
)
)
GROUP BY groupby_only_col
),
long_array_aggregated AS (
 SELECT index,
 ARRAY_AGG(value ORDER BY rank) AS value
FROM (
SELECT ranked_classes_by_id.index AS index,
ranked_classes_by_id.rank as rank,
source.value as value
FROM `project.dataset.Table` as source
RIGHT JOIN (
SELECT index,
 class,
rank() over (PARTITION BY index ORDER BY class as rank
FROM (
 SELECT DISTINCT class
FROM `project.dataset.Table`)
 CROSS JOIN (
 SELECT DISTINCT index
FROM `project.dataset.Table`)
) as ranked_classes_by_id
 USING(index, class)
)
GROUP BY index
)
SELECT long_array_aggregated.index, 
Long_array_aggregated.value[ordinal(A)] as A,
Long_array_aggregated

In [47]:
pivot_col_vals = ['a','b']
pivot_col = 'class'
"".join(["ANY_VALUE(IF({} = '{}', rank, null)) as {}".format(pivot_col, pivot_col_val, pivot_col_val) 
                    for pivot_col_val in pivot_col_vals])
    

"ANY_VALUE(IF(red = 'a', rank, null)) as aANY_VALUE(IF(red = 'b', rank, null)) as b"

In [12]:
print(wide_ranked_query + long_array_agg)

WITH wide_ranked AS ( 
SELECT ANY_VALUE(IF(class = 'A', rank, null)) as A,
ANY_VALUE(IF(class = 'B', rank, null)) as B,
ANY_VALUE(IF(class = 'C', rank, null)) as C,
ANY_VALUE(IF(class = 'D', rank, null)) as D
FROM (
SELECT "1" AS groupby_only_col,
class,
RANK() over (ORDER BY class) AS rank
FROM (
SELECT DISTINCT class
FROM `project.dataset.Table`
)
)
GROUP BY groupby_only_col
),
long_array_aggregated AS (
 SELECT class,
 ARRAY_AGG(value ORDER BY rank) AS values
FROM (
SELECT ranked_classes_by_id.class AS class,
ranked_classes_by_id.rank as rank,
source.values as values
FROM `project.dataset.Table` as source
RIGHT JOIN (
SELECT class,
 ['A', 'B', 'C', 'D'],
rank() over (PARTITION BY class ORDER BY ['A', 'B', 'C', 'D'] as rank
FROM (
 SELECT DISTINCT class
FROM `project.dataset.Table`)
 CROSS JOIN (
 SELECT DISTINCT class
FROM `project.dataset.Table`)
) as ranked_classes_by_id
 USING(class, ['A', 'B', 'C', 'D'])
)GROUP BY class
)


In [106]:
import re

import pandas as pd


class BqPivot():
    """
    Class to generate a SQL query which creates pivoted tables in BigQuery.
    Example
    -------
    The following example uses the kaggle's titanic data. It can be found here -
    `https://www.kaggle.com/c/titanic/data`
    This data is only 60 KB and it has been used for a demonstration purpose.
    This module comes particularly handy with huge datasets for which we would need
    BigQuery(https://en.wikipedia.org/wiki/BigQuery).
    >>> from bq_pivot import BqPivot
    >>> import pandas as pd
    >>> data = pd.read_csv("titanic.csv").head()
    >>> gen = BqPivot(data=data, index_col=["Pclass", "Survived", "PassengenId"],
                      pivot_col="Name", values_col="Age",
                      add_col_nm_suffix=False)
    >>> print(gen.generate_query())
    select Pclass, Survived, PassengenId, 
    sum(case when Name = "Braund, Mr. Owen Harris" then Age else 0 end) as braund_mr_owen_harris,
    sum(case when Name = "Cumings, Mrs. John Bradley (Florence Briggs Thayer)" then Age else 0 end) as cumings_mrs_john_bradley_florence_briggs_thayer,
    sum(case when Name = "Heikkinen, Miss. Laina" then Age else 0 end) as heikkinen_miss_laina,
    sum(case when Name = "Futrelle, Mrs. Jacques Heath (Lily May Peel)" then Age else 0 end) as futrelle_mrs_jacques_heath_lily_may_peel,
    sum(case when Name = "Allen, Mr. William Henry" then Age else 0 end) as allen_mr_william_henry
    from <--insert-table-name-here-->
    group by 1,2,3
    
    """
    def __init__(self, index_col, pivot_col, values_col, agg_fun="sum", data=None, 
                 table_name=None, not_eq_default="0", add_col_nm_suffix=True, custom_agg_fun=None,
                 prefix=None, suffix=None):
        """
        Parameters
        ----------
        data: pandas.core.frame.DataFrame or string
            The input data can either be a pandas dataframe or a string path to the pandas
            data frame. The only requirement of this data is that it must have the column
            on which the pivot it to be done.
        index_col: list
            The names of the index columns in the query (the columns on which the group by needs to be performed)
        pivot_col: string
            The name of the column on which the pivot needs to be done.
        values_col: string or list of strings
            The name or names of the columns on which aggregation needs to be performed.
        agg_fun: string
            The name of the sql aggregation function.
        table_name: string
            The name of the table in the query.
        not_eq_default: numeric, optional
            The value to take when the case when statement is not satisfied. For example,
            if one is doing a sum aggregation on the value column then the not_eq_default should
            be equal to 0. Because the case statement part of the sql query would look like - 
            
            ... ...
            sum(case when <pivot_col> = <some_pivot_col_value> then values_col else 0)
            ... ...
            Similarly if the aggregation function is min then the not_eq_default should be
            positive infinity.
        add_col_nm_suffix: boolean, optional
            If True, then the original values column name will be added as suffix in the new 
            pivoted columns.
        custom_agg_fun: string, optional
            Can be used if one wants to give customized aggregation function. The values col name 
            should be replaced with {}. For example, if we want an aggregation function like - 
            sum(coalesce(values_col, 0)) then the custom_agg_fun argument would be - 
            sum(coalesce({}, 0)). 
            If provided this would override the agg_fun argument.
        prefix: string, optional
            A fixed string to add as a prefix in the pivoted column names separated by an
            underscore.
        suffix: string, optional
            A fixed string to add as a suffix in the pivoted column names separated by an
            underscore.        
        """
        if data == None and table_name == None:
            raise ValueError("At least one of data or table_name must be provided.")
            
        self.query = ""

        self.index_col = index_col
        self.values_col = values_col
        self.pivot_col = pivot_col

        self.not_eq_default = not_eq_default
        
        
        if table_name:
            self.table_name = table_name
            self.piv_col_vals = self._query_piv_col_vals()
        elif data:
            self.piv_col_vals = self._get_piv_col_vals(data)
            self.table_name = self._get_table_name(table_name)
            
        if type(self.values_col) == str:
            self.piv_col_names = self._create_piv_col_names(add_col_nm_suffix, prefix, suffix)
        elif type(self.values_col) == list:
            self.piv_col_names = []
            for value_col in self.values_col:
                self.piv_col_names.append(self._create_piv_col_names(add_col_nm_suffix, prefix, suffix, value_col))
        self.ord_col_names = self._create_ord_col_names()
        
        self.function = custom_agg_fun if custom_agg_fun else agg_fun + "({})"

    def _get_table_name(self, table_name):
        """
        Returns the table name or a placeholder if the table name is not provided.
        """
        return table_name if table_name else "<--insert-table-name-here-->"
    
    def _query_piv_col_vals(self):
        return pd.read_gbq(f'SELECT DISTINCT({self.pivot_col}) FROM {self.table_name}')[self.pivot_col].astype(str).to_list()
    

    def _get_piv_col_vals(self, data):
        """
        Gets all the unique values of the pivot column.
        """
        if isinstance(data, pd.DataFrame):
            self.data = data
        elif isinstance(data, str):
            self.data = pd.read_csv(data)
        else:
            raise ValueError("Provided data must be a pandas dataframe or a csv file path.")

        if self.pivot_col not in self.data.columns:
            raise ValueError("The provided data must have the column on which pivot is to be done. "\
                             "Also make sure that the column name in the data is same as the name "\
                             "provided to the pivot_col parameter.")

        return self.data[self.pivot_col].astype(str).unique().tolist()
    
    def _clean_col_name(self, col_name):
        """
        The pivot column values can have arbitrary strings but in order to 
        convert them to column names some cleaning is required. This method 
        takes a string as input and returns a clean column name.
        """
        
        # replace spaces with underscores
        # remove non alpha numeric characters other than underscores
        # replace multiple consecutive underscores with one underscore
        # make all characters lower case
        # remove trailing underscores
        return re.sub("_+", "_", re.sub('[^0-9a-zA-Z_]+', '', re.sub(" ", "_", col_name))).lower().rstrip("_")

    def _create_piv_col_names(self, add_col_nm_suffix, prefix, suffix, value_col=None):
        """
        The method created a list of pivot column names of the new pivoted table.
        """
        prefix = prefix + "_" if prefix else ""
        suffix = "_" + suffix if suffix else ""
        if value_col == None:
            value_col = self.values_col

        if add_col_nm_suffix:
            piv_col_names = ["{0}{1}_{2}{3}".format(prefix, self._clean_col_name(piv_col_val), value_col.lower(), suffix)
                             for piv_col_val in self.piv_col_vals]
        else:
            piv_col_names = ["{0}{1}{2}".format(prefix, self._clean_col_name(piv_col_val), suffix)
                             for piv_col_val in self.piv_col_vals]

        return piv_col_names
    
    def _create_ord_col_names(self):
        '''
        Create sanitized base ordinal names for each piv_col_val.
        '''
        
        ord_col_names = ["{}_".format(self._clean_col_name(piv_col_val))
                             for piv_col_val in self.piv_col_vals]
        return ord_col_names
        

    def _write_wide_ranked(self):
        '''
        Writes a 'wide_ranked' table
        
        To do: substitue the current piv_col_names for ordinal_piv_col_names like "ALL_" instead of "ALL_size"
        This also applies to _write_table_join in the ordinal() call 
        '''
        
        query = 'WITH wide_ranked AS ( \nSELECT '
        query = query + "".join(["ANY_VALUE(IF({} = '{}', rank, null)) as {},\n".format(self.pivot_col, 
                                                                                        pivot_col_val, 
                                                                                        ord_col_name) 
                                    for pivot_col_val, ord_col_name in zip(self.piv_col_vals, self.ord_col_names)])

        query = query[:-2] +'\nFROM (\nSELECT "1" AS groupby_only_col,\n'
        query = query + f"{self.pivot_col},\nRANK() over (ORDER BY {self.pivot_col}) AS rank\nFROM (\nSELECT DISTINCT {self.pivot_col}\n"
        query = query + f"FROM `{self.table_name}`\n)\n)\nGROUP BY groupby_only_col\n),\n"

        return query 


    def _write_long_array_aggregated(self):
        query = ""
        
        # replace all self.values_col with value_col, modify table names with i
        query = query + f"long_array_aggregated AS (\n SELECT {self.index_col},\n "
        query = query + "".join([f"ARRAY_AGG({values_col} ORDER BY rank) AS {values_col},\n" for values_col in self.values_col])[:-2]
        query = query + f"\nFROM (\nSELECT ranked_classes_by_id.{self.index_col} AS {self.index_col},\n"
        query = query + f"ranked_classes_by_id.rank as rank,\n"
        query = query + "".join([f"source.{values_col} as {values_col},\n" for values_col in self.values_col])[:-2]
        query = query + f"\n FROM `{self.table_name}` as source\n"
        query = query + f"RIGHT JOIN (\nSELECT {self.index_col},\n {self.pivot_col},\n"
        query = query + f"rank() over (PARTITION BY {self.index_col} ORDER BY {self.pivot_col}) as rank\n"
        query = query + f"FROM (\n SELECT DISTINCT {self.pivot_col}\n"
        query = query + f"FROM `{self.table_name}`)\n CROSS JOIN (\n SELECT DISTINCT {self.index_col}\n"
        query = query + f"FROM `{self.table_name}`)\n"
        query = query + f") as ranked_classes_by_id\n USING({self.index_col}, {self.pivot_col})\n)\nGROUP BY {self.index_col}\n)\n"

        return query

    def _write_table_join(self):
        
        query = f"SELECT long_array_aggregated.{self.index_col}, \n"
        
        for i, values_col in enumerate(self.values_col):
            query = query + "".join(["Long_array_aggregated.{}[ordinal({})] as {},\n".format(values_col, ord_col_name, pivot_col_name)  
                                                  for ord_col_name, pivot_col_name in zip(self.ord_col_names, self.piv_col_names[i])]) #values col 

        #f"long_array_aggregated.{value_col}[ordinal({value for value in values_col})] as {value for value in values_col}, \n"

        query = query[:-2] + f"\nfrom "
#         for values_col in [self.values_col]:
        query = query + "long_array_aggregated, "
            
        query = query + "wide_ranked"
        return query
    
    
    
    # replace with new query functions 
    def generate_query(self):
        """
        Returns the query to create the pivoted table.
        
        In order to do this operation for multiple columns, we now need to iterate over the _write_long_array_aggregated function
        Or, inside long array aggregated we need to iterate over the entire block 
        """
        self.query = self._write_wide_ranked() +\
                     self._write_long_array_aggregated() +\
                     self._write_table_join() 

        return self.query

    def write_query(self, output_file=None):
        """
        Writes the query to a text file if output_file is passed, or prints the query to the console.
        """
        if output_file == None:
            print(self.generate_query())
        else:
            text_file = open(output_file, "w")
            text_file.write(self.generate_query())
            text_file.close()
            
    def submit_query(self, **kwargs):
        '''
        Submits the query and returns the results. 
        '''
        if self.query == "":
            self.generate_query()
        return pd.read_gbq(self.query)
    


In [109]:
gen = BqPivot(table_name='sp500historical.agg_1mm_bars',
              index_col="interval_range",
              pivot_col="SYMBOL",
              values_col=["size","open","close"])

In [110]:
gen.write_query(output_file='./new_query_test.txt')

In [40]:
df = gen.submit_query()

GenericGBQException: Reason: 400 Syntax error: Unexpected keyword ALL at [80:46]

(job ID: 1546de2e-51f3-4bee-9358-152903f6438a)

                   -----Query Job SQL Follows-----                   

    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:WITH wide_ranked AS ( 
   2:SELECT ANY_VALUE(IF(SYMBOL = 'A', rank, null)) as A,
   3:ANY_VALUE(IF(SYMBOL = 'C', rank, null)) as C,
   4:ANY_VALUE(IF(SYMBOL = 'D', rank, null)) as D,
   5:ANY_VALUE(IF(SYMBOL = 'F', rank, null)) as F,
   6:ANY_VALUE(IF(SYMBOL = 'J', rank, null)) as J,
   7:ANY_VALUE(IF(SYMBOL = 'K', rank, null)) as K,
   8:ANY_VALUE(IF(SYMBOL = 'L', rank, null)) as L,
   9:ANY_VALUE(IF(SYMBOL = 'O', rank, null)) as O,
  10:ANY_VALUE(IF(SYMBOL = 'T', rank, null)) as T,
  11:ANY_VALUE(IF(SYMBOL = 'V', rank, null)) as V,
  12:ANY_VALUE(IF(SYMBOL = 'BA', rank, null)) as BA,
  13:ANY_VALUE(IF(SYMBOL = 'BK', rank, null)) as BK,
  14:ANY_VALUE(IF(SYMBOL = 'BR', rank, null)) as BR,
  15:ANY_VALUE(IF(SYMBOL = 'C

In [ ]:
List of fixes 

- ALL column is reserved
- BF.B, BRK.B -> BFB, BRKB


In [111]:
df = pd.read_csv('./results-20200925-154137.csv')

In [114]:
df.sort_values(by='interval_range', ascending=False)

,interval_range,a_size,c_size,d_size,f_size,j_size,k_size,l_size,o_size,t_size,...,wynn_close,xlnx_close,xray_close,zbra_close,zion_close,brkb_close,cmcsa_close,disca_close,disck_close,googl_close
656,"(98000000000.0, 99000000000.0]",5796.0,105914.0,55634.0,179023.0,3551.0,12364.0,2344.0,7196.0,137429.0,...,83.9698,96.4450,44.7500,262.06,35.8200,181.18,39.491,22.2400,20.21,1470.1188
547,"(97000000000.0, 98000000000.0]",6709.0,85882.0,69622.0,307749.0,2757.0,10316.0,3280.0,11414.0,106703.0,...,83.9479,96.4447,44.6150,262.05,35.7965,181.03,39.450,22.2550,20.21,1468.7405
316,"(966000000000.0, 967000000000.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"(96000000000.0, 97000000000.0]",7501.0,118423.0,84007.0,259948.0,3984.0,18531.0,4577.0,15048.0,199871.0,...,84.0097,96.4400,44.5832,262.06,35.7900,181.04,39.500,22.2500,20.20,1468.6400
109,"(952000000000.0, 953000000000.0]",11684.0,9443.0,1756.0,39020.0,670.0,3684.0,1770.0,4363.0,26414.0,...,69.0500,91.0200,43.1200,246.31,32.2800,175.57,38.670,20.2597,18.50,1362.1400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,"(1070000000000.0, 1071000000000.0]",1907029.0,103.0,NaN,113.0,NaN,NaN,NaN,NaN,2.0,...,69.0000,NaN,NaN,NaN,NaN,175.82,NaN,NaN,NaN,1365.0000
457,"(1064000000000.0, 1065000000000.0]",NaN,922.0,NaN,5550.0,NaN,NaN,NaN,NaN,1501.0,...,69.9000,NaN,NaN,NaN,NaN,175.37,38.650,NaN,NaN,1365.4400
344,"(1062000000000.0, 1063000000000.0]",NaN,23.0,NaN,705.0,NaN,1080.0,26877.0,NaN,13.0,...,69.8500,91.0200,NaN,NaN,NaN,175.37,38.650,NaN,NaN,1364.0399
361,"(1052000000000.0, 1053000000000.0]",NaN,NaN,NaN,6.0,NaN,2900.0,NaN,22860.0,NaN,...,NaN,91.0200,43.1200,246.42,32.2900,175.37,38.650,NaN,NaN,1361.0400


In [121]:

class BqPivot():
    """
    Class to generate a SQL query which creates pivoted tables in BigQuery.
    Example
    -------
    The following example uses the kaggle's titanic data. It can be found here -
    `https://www.kaggle.com/c/titanic/data`
    This data is only 60 KB and it has been used for a demonstration purpose.
    This module comes particularly handy with huge datasets for which we would need
    BigQuery(https://en.wikipedia.org/wiki/BigQuery).
    >>> from bq_pivot import BqPivot
    >>> import pandas as pd
    >>> data = pd.read_csv("titanic.csv").head()
    >>> gen = BqPivot(data=data, index_col=["Pclass", "Survived", "PassengenId"],
                      pivot_col="Name", values_col="Age",
                      add_col_nm_suffix=False)
    >>> print(gen.generate_query())
    select Pclass, Survived, PassengenId,
    sum(case when Name = "Braund, Mr. Owen Harris" then Age else 0 end) as braund_mr_owen_harris,
    sum(case when Name = "Cumings, Mrs. John Bradley (Florence Briggs Thayer)" then Age else 0 end) as cumings_mrs_john_bradley_florence_briggs_thayer,
    sum(case when Name = "Heikkinen, Miss. Laina" then Age else 0 end) as heikkinen_miss_laina,
    sum(case when Name = "Futrelle, Mrs. Jacques Heath (Lily May Peel)" then Age else 0 end) as futrelle_mrs_jacques_heath_lily_may_peel,
    sum(case when Name = "Allen, Mr. William Henry" then Age else 0 end) as allen_mr_william_henry
    from <--insert-table-name-here-->
    group by 1,2,3

    """

    def __init__(self, index_col, pivot_col, values_col, table_name=None, data=None,
                 agg_fun="sum", not_eq_default="0", add_col_nm_suffix=True, custom_agg_fun=None,
                 prefix=None, suffix=None):
        """
        Parameters
        ----------
        index_col: list
            The names of the index columns in the query (the columns on which the group by needs to be performed)
        pivot_col: string
            The name of the column on which the pivot needs to be done.
        values_col: string or list of strings
            The name or names of the columns on which aggregation needs to be performed.
        agg_fun: string
            The name of the sql aggregation function.
        data: pandas.core.frame.DataFrame or string
            The input data can either be a pandas dataframe or a string path to the pandas
            data frame. The only requirement of this data is that it must have the column
            on which the pivot it to be done. If data is not provided the __init__ call will
            automatically query the table_name provided to get distinct pivot column values.
            **Must provide one of data or table_name**
        table_name: string
            The name of the table in the query.
            **Must provide one of data or table_name**
        not_eq_default: numeric, optional
            The value to take when the case when statement is not satisfied. For example,
            if one is doing a sum aggregation on the value column then the not_eq_default should
            be equal to 0. Because the case statement part of the sql query would look like -

            ... ...
            sum(case when <pivot_col> = <some_pivot_col_value> then values_col else 0)
            ... ...
            Similarly if the aggregation function is min then the not_eq_default should be
            positive infinity.
        add_col_nm_suffix: boolean, optional
            If True, then the original values column name will be added as suffix in the new
            pivoted columns.
        custom_agg_fun: string, optional
            Can be used if one wants to give customized aggregation function. The values col name
            should be replaced with {}. For example, if we want an aggregation function like -
            sum(coalesce(values_col, 0)) then the custom_agg_fun argument would be -
            sum(coalesce({}, 0)).
            If provided this would override the agg_fun argument.
        prefix: string, optional
            A fixed string to add as a prefix in the pivoted column names separated by an
            underscore.
        suffix: string, optional
            A fixed string to add as a suffix in the pivoted column names separated by an
            underscore.
        """
        if data == None and table_name == None:
            raise ValueError("At least one of data or table_name must be provided.")

        self.query = ""

        self.index_col = index_col
        self.values_col = values_col
        self.pivot_col = pivot_col

        self.not_eq_default = not_eq_default

        if data is None:
            self.table_name = table_name
            self.piv_col_vals = self._query_piv_col_vals()
        elif data:
            self.piv_col_vals = self._get_piv_col_vals(data)
            self.table_name = self._get_table_name(table_name)

        if type(self.values_col) == str:
            self.piv_col_names = self._create_piv_col_names(add_col_nm_suffix, prefix, suffix)
        elif type(self.values_col) == list:
            self.piv_col_names = []
            for value_col in self.values_col:
                self.piv_col_names.append(self._create_piv_col_names(add_col_nm_suffix, prefix, suffix, value_col))

        self.ord_col_names = self._create_ord_col_names()

        self.function = custom_agg_fun if custom_agg_fun else agg_fun + "({})"

    def _get_table_name(self, table_name):
        """
        Returns the table name or a placeholder if the table name is not provided.
        """
        return table_name if table_name else "<--insert-table-name-here-->"

    def _query_piv_col_vals(self):
        '''
        Queries the distinct values in the pivot col directly from the table_name provided.
        '''
        return pd.read_gbq(f'SELECT DISTINCT({self.pivot_col}) FROM {self.table_name}')[self.pivot_col].astype(
            str).to_list()

    def _get_piv_col_vals(self, data):
        """
        Gets all the unique values of the pivot column.
        """
        if isinstance(data, pd.DataFrame):
            self.data = data
        elif isinstance(data, str):
            self.data = pd.read_csv(data)
        else:
            raise ValueError("Provided data must be a pandas dataframe or a csv file path.")

        if self.pivot_col not in self.data.columns:
            raise ValueError("The provided data must have the column on which pivot is to be done. " \
                             "Also make sure that the column name in the data is same as the name " \
                             "provided to the pivot_col parameter.")

        return self.data[self.pivot_col].astype(str).unique().tolist()

    def _clean_col_name(self, col_name):
        """
        The pivot column values can have arbitrary strings but in order to
        convert them to column names some cleaning is required. This method
        takes a string as input and returns a clean column name.
        """

        # replace spaces with underscores
        # remove non alpha numeric characters other than underscores
        # replace multiple consecutive underscores with one underscore
        # make all characters lower case
        # remove trailing underscores
        return re.sub("_+", "_", re.sub('[^0-9a-zA-Z_]+', '', re.sub(" ", "_", col_name))).lower().rstrip("_")

    def _create_piv_col_names(self, add_col_nm_suffix, prefix, suffix, value_col=None):
        """
        The method created a list of pivot column names of the new pivoted table.
        """
        prefix = prefix + "_" if prefix else ""
        suffix = "_" + suffix if suffix else ""
        if value_col == None:
            value_col = self.values_col

        if add_col_nm_suffix:
            piv_col_names = [
                "{0}{1}_{2}{3}".format(prefix, self._clean_col_name(piv_col_val), value_col.lower(), suffix)
                for piv_col_val in self.piv_col_vals]
        else:
            piv_col_names = ["{0}{1}{2}".format(prefix, self._clean_col_name(piv_col_val), suffix)
                             for piv_col_val in self.piv_col_vals]

        return piv_col_names

    def _create_ord_col_names(self):
        '''
        Create sanitized base ordinal names for each piv_col_val.
        '''

        ord_col_names = ["{}_".format(self._clean_col_name(piv_col_val))
                         for piv_col_val in self.piv_col_vals]
        return ord_col_names

    def _write_wide_ranked(self):
        '''
        Writes a 'wide_ranked' table

        To do: substitue the current piv_col_names for ordinal_piv_col_names like "ALL_" instead of "ALL_size"
        This also applies to _write_table_join in the ordinal() call
        '''

        query = 'WITH wide_ranked AS ( \nSELECT '
        query = query + "".join(["ANY_VALUE(IF({} = '{}', rank, null)) as {},\n".format(self.pivot_col,
                                                                                        pivot_col_val,
                                                                                        ord_col_name)
                                 for pivot_col_val, ord_col_name in zip(self.piv_col_vals, self.ord_col_names)])

        query = query[:-2] + '\nFROM (\nSELECT "1" AS groupby_only_col,\n'
        query = query + f"{self.pivot_col},\nRANK() over (ORDER BY {self.pivot_col}) AS rank\nFROM (\nSELECT DISTINCT {self.pivot_col}\n"
        query = query + f"FROM `{self.table_name}`\n)\n)\nGROUP BY groupby_only_col\n),\n"

        return query

    def _write_long_array_aggregated(self):
        query = ""

        # replace all self.values_col with value_col, modify table names with i
        query = query + f"long_array_aggregated AS (\n SELECT {self.index_col},\n "
        query = query + "".join(
            [f"ARRAY_AGG({values_col} ORDER BY rank) AS {values_col},\n" for values_col in self.values_col])[:-2]
        query = query + f"\nFROM (\nSELECT ranked_classes_by_id.{self.index_col} AS {self.index_col},\n"
        query = query + f"ranked_classes_by_id.rank as rank,\n"
        query = query + "".join([f"source.{values_col} as {values_col},\n" for values_col in self.values_col])[:-2]
        query = query + f"\n FROM `{self.table_name}` as source\n"
        query = query + f"RIGHT JOIN (\nSELECT {self.index_col},\n {self.pivot_col},\n"
        query = query + f"rank() over (PARTITION BY {self.index_col} ORDER BY {self.pivot_col}) as rank\n"
        query = query + f"FROM (\n SELECT DISTINCT {self.pivot_col}\n"
        query = query + f"FROM `{self.table_name}`)\n CROSS JOIN (\n SELECT DISTINCT {self.index_col}\n"
        query = query + f"FROM `{self.table_name}`)\n"
        query = query + f") as ranked_classes_by_id\n USING({self.index_col}, {self.pivot_col})\n)\nGROUP BY {self.index_col}\n)\n"

        return query

    def _write_table_join(self):

        query = f"SELECT long_array_aggregated.{self.index_col}, \n"

        for i, values_col in enumerate(self.values_col):
            query = query + "".join(
                ["Long_array_aggregated.{}[ordinal({})] as {},\n".format(values_col, ord_col_name, pivot_col_name)
                 for ord_col_name, pivot_col_name in zip(self.ord_col_names, self.piv_col_names[i])])  # values col

        # f"long_array_aggregated.{value_col}[ordinal({value for value in values_col})] as {value for value in values_col}, \n"

        query = query[:-2] + f"\nfrom "
        #         for values_col in [self.values_col]:
        query = query + "long_array_aggregated, "

        query = query + "wide_ranked"
        return query

    # replace with new query functions
    def generate_query(self):
        """
        Returns the query to create the pivoted table.

        In order to do this operation for multiple columns, we now need to iterate over the _write_long_array_aggregated function
        Or, inside long array aggregated we need to iterate over the entire block
        """
        self.query = self._write_wide_ranked() + \
                     self._write_long_array_aggregated() + \
                     self._write_table_join()

        return self.query

    def write_query(self, output_file=None, verbose=False):
        """
        Writes the query to a text file if output_file is passed, or prints the query to the console.
        """
        self.generate_query()
        if verbose:
            print(self.query)
        if output_file is not None:
            text_file = open(output_file, "w")
            text_file.write(self.generate_query())
            text_file.close()

    def submit_pandas_query(self, **kwargs):
        '''
        Submits the query and returns the results.
        '''
        if self.query == "":
            self.generate_query()
        return pd.read_gbq(self.query)

    def write_permanent_table(self, destination_table):
        job_config = bigquery.QueryJobConfig(
                                            allow_large_results=True,
                                            destination=destination_table,
                                            use_legacy_sql=True
                                                                )
        if self.query == "":
            self.generate_query()
        sql = self.query()

        # Start the query, passing in the extra configuration.
        query_job = client.query(sql, job_config=job_config)  # Make an API request.
        query_job.result()  # Wait for the job to complete.

        print("Query results loaded to the table {}".format(table_id))


In [122]:
gen = BqPivot(table_name='sp500historical.agg_1mm_bars',
              index_col="interval_range",
              pivot_col="SYMBOL",
              values_col=["size","open","close"])

In [123]:
df = gen.submit_pandas_query()

In [129]:
import pandas as pd
str_intervals = [i.replace("(","").replace("]", "").split(", ") for i in df['interval_range']]
original_cuts = [pd.Interval(float(i), float(j)) for i, j in str_intervals]
df['interval_range'] = original_cuts

In [130]:
df = df.sort_values(by='interval_range',ascending=True)

In [132]:
df.head(10)

,interval_range,a_size,c_size,d_size,f_size,j_size,k_size,l_size,o_size,t_size,...,wynn_close,xlnx_close,xray_close,zbra_close,zion_close,brkb_close,cmcsa_close,disca_close,disck_close,googl_close
1189,"(0.0, 1000000000.0]",6.0,88719.0,NaN,319154.0,NaN,450.0,15.0,571.0,79878.0,...,84.8200,95.690,NaN,NaN,NaN,182.00,38.970,21.90,NaN,1459.0000
2022,"(1000000000.0, 2000000000.0]",17.0,182080.0,176.0,313757.0,NaN,285.0,765.0,5643.0,149337.0,...,84.3100,95.500,44.000,262.7500,NaN,182.01,38.850,22.27,NaN,1455.8200
1838,"(2000000000.0, 3000000000.0]",NaN,28372.0,15.0,63661.0,352.0,500.0,7.0,1764.0,55675.0,...,84.6100,95.190,NaN,263.1973,36.39,181.99,38.880,22.20,20.160,1455.9900
2641,"(3000000000.0, 4000000000.0]",33828.0,NaN,43769.0,50.0,10854.0,NaN,20934.0,NaN,NaN,...,NaN,95.020,44.330,NaN,NaN,NaN,38.850,NaN,20.010,NaN
3226,"(4000000000.0, 5000000000.0]",NaN,648581.0,442.0,1473674.0,NaN,35794.0,1.0,62836.0,1168728.0,...,84.5250,95.020,NaN,NaN,NaN,NaN,38.960,NaN,20.160,1455.0600
1017,"(5000000000.0, 6000000000.0]",NaN,7996.0,153.0,16649.0,NaN,NaN,NaN,3.0,6466.0,...,84.5100,95.395,44.400,266.1500,NaN,NaN,38.800,NaN,NaN,1455.0000
2782,"(6000000000.0, 7000000000.0]",2.0,45582.0,1777.0,417820.0,17.0,3045.0,105.0,5364.0,75613.0,...,84.1819,95.398,44.200,264.8210,36.76,181.76,38.730,22.16,20.075,1454.8676
834,"(7000000000.0, 8000000000.0]",157.0,94371.0,3656.0,616640.0,208.0,2217.0,22.0,1627.0,90811.0,...,84.4000,95.500,44.520,264.8000,36.55,181.62,38.985,22.36,20.240,1453.7300
1,"(8000000000.0, 9000000000.0]",7.0,105997.0,2862.0,658455.0,2.0,4788.0,624.0,4148.0,231207.0,...,83.9800,95.430,44.275,265.9600,36.43,181.63,38.990,22.47,20.280,1453.2300
206,"(9000000000.0, 10000000000.0]",221.0,76698.0,987.0,303595.0,104.0,989.0,990.0,6163.0,93158.0,...,83.4700,95.545,44.500,264.9000,36.24,181.43,38.745,22.38,20.260,1450.3150
